# 0. Install and Import Dependencies

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

  Using cached tensorflow-2.4.1-cp38-cp38-macosx_10_11_x86_64.whl (173.9 MB)
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.4.1 (from versions: none)
ERROR: No matching distribution found for tensorflow-gpu==2.4.1


In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

# 1. Load Model

In [2]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [5]:
cap = cv2.VideoCapture(0)

WINDOW_WIDTH = int(cap.get(3))
WINDOW_HEIGHT = int(cap.get(4))
print(WINDOW_WIDTH, WINDOW_HEIGHT)

1280 720


In [6]:
error = False
    

In [7]:


def checkPositions(keypoints_with_scores, frame):
    nose = np.array(keypoints_with_scores[0][0][0][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_eye = np.array(keypoints_with_scores[0][0][1][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_eye = np.array(keypoints_with_scores[0][0][2][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_ear = np.array(keypoints_with_scores[0][0][3][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_ear = np.array(keypoints_with_scores[0][0][4][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_shoulder = np.array(keypoints_with_scores[0][0][5][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_shoulder = np.array(keypoints_with_scores[0][0][6][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_elbow = np.array(keypoints_with_scores[0][0][7][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_elbow = np.array(keypoints_with_scores[0][0][8][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_wrist = np.array(keypoints_with_scores[0][0][9][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_wrist = np.array(keypoints_with_scores[0][0][10][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_hip = np.array(keypoints_with_scores[0][0][11][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_hip = np.array(keypoints_with_scores[0][0][12][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_knee = np.array(keypoints_with_scores[0][0][13][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_knee = np.array(keypoints_with_scores[0][0][14][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    left_ankle = np.array(keypoints_with_scores[0][0][15][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    right_ankle = np.array(keypoints_with_scores[0][0][16][:2]*[WINDOW_HEIGHT, WINDOW_WIDTH]).astype(int)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    coordinates = "(" + str(left_hip[1]) + ", " + str(left_hip[0]) + ")"
    cv2.putText(frame, coordinates, (50, 50), font, 1, (255, 0, 0), 2, cv2.LINE_4)
    
    color = (0, 255, 255)
    if(left_hip[1] < int(WINDOW_WIDTH / 3) and int((WINDOW_WIDTH / 3) * 2) > left_hip[1]):
        color = (255, 0, 0)
    
    cv2.line(frame, (int(WINDOW_WIDTH / 3), 0), (int(WINDOW_WIDTH / 3), WINDOW_HEIGHT), color, 2)
    cv2.line(frame, (int((WINDOW_WIDTH / 3) * 2), 0), (int((WINDOW_WIDTH / 3) * 2), WINDOW_HEIGHT), color, 2)
        


In [8]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

# 2. Make Detections

In [9]:
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    right_eye = keypoints_with_scores[0][0][1]
    
    # Rendering 
    #draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    checkPositions(keypoints_with_scores, frame)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# 3. Draw Keypoints

# 4. Draw Edges

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [5]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)